# 使用嵌入式模型和适配网络进行文本分类

In [5]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
from sentence_transformers import SentenceTransformer

In [6]:
from datasets import load_dataset
data = load_dataset("rotten_tomatoes", cache_dir="./cache")

In [4]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2', cache_folder="./model_cache")

modules.json: 0.00B [00:00, ?B/s]

d:\Software\miniconda3\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\liuzhiqiang\repos\auto_devops\model_cache\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

In [7]:
train_embeddings = model.encode(data["train"]["text"], show_progress_bar=True)
test_embeddings = model.encode(data["test"]["text"], show_progress_bar=True)
# 768维嵌入向量
train_embeddings.shape

Batches:   0%|          | 0/267 [00:00<?, ?it/s]

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

(8530, 768)

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def evaluate_performance(y_true, y_pred):
    performance = classification_report(y_true, y_pred, target_names=["negative", "positive"])
    print(performance)

In [9]:
clf = LogisticRegression(random_state=42)
clf.fit(train_embeddings, data["train"]["label"])

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [11]:
y_pred = clf.predict(test_embeddings)
evaluate_performance(data["test"]["label"], y_pred)

              precision    recall  f1-score   support

    negative       0.85      0.86      0.85       533
    positive       0.86      0.85      0.85       533

    accuracy                           0.85      1066
   macro avg       0.85      0.85      0.85      1066
weighted avg       0.85      0.85      0.85      1066



# 零样本嵌入式预测

对类别进行描述，如“一条负面影评”，通过嵌入模型生成嵌入向量，然后对文档生成嵌入向量，比较余弦相似度。

In [ ]:
# 描述信息的定义具有决定性作用
label_embeddings = model.encode(["A very negative review", "A very positive review"])
#label_embeddings = model.encode(["A very negative review", "A very positive review"])

In [37]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
# 为每个文档找到最匹配的标签
sim_matrix = cosine_similarity(test_embeddings, label_embeddings)
y_pred = np.argmax(sim_matrix, axis=1)

In [38]:
evaluate_performance(data["test"]["label"], y_pred)

              precision    recall  f1-score   support

    negative       0.73      0.82      0.77       533
    positive       0.80      0.70      0.75       533

    accuracy                           0.76      1066
   macro avg       0.76      0.76      0.76      1066
weighted avg       0.76      0.76      0.76      1066

